# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import re
import pickle
import warnings
warnings.filterwarnings("ignore")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
MOC_RF = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MOC_RF)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train.values, y_train.values)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
def get_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', cat)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

In [9]:
results = get_results(y_test, y_pred)
results


Aggregated f_score: 0.927246421353
Aggregated precision: 0.93099853365
Aggregated recall: 0.941456447293


,Category,f_score,precision,recall
1,related,0.761806,0.761162,0.78578
2,request,0.857445,0.87072,0.877327
3,offer,0.994511,0.99269,0.996338
4,aid_related,0.718903,0.735506,0.730699
5,medical_help,0.895452,0.89545,0.922338
6,medical_products,0.930269,0.936404,0.951327
7,search_and_rescue,0.958078,0.972426,0.97162
8,security,0.971483,0.962219,0.980928
9,military,0.953477,0.957424,0.967501
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f653c9b56a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
parameters = {'clf__estimator__max_depth': [10, 50,  None],
              'clf__estimator__min_samples_leaf':[2, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train.values, y_train.values)
y_pred = cv.predict(X_test)
results2 = get_results(y_test, y_pred)
results2


Aggregated f_score: 0.922546219528
Aggregated precision: 0.92893594873
Aggregated recall: 0.940350252602


,Category,f_score,precision,recall
1,related,0.751587,0.781676,0.796308
2,request,0.840372,0.868568,0.869393
3,offer,0.994511,0.99269,0.996338
4,aid_related,0.743865,0.747705,0.748703
5,medical_help,0.890383,0.896944,0.922643
6,medical_products,0.928461,0.937337,0.951022
7,search_and_rescue,0.957255,0.957715,0.971163
8,security,0.971483,0.962219,0.980928
9,military,0.952688,0.953676,0.967043
10,child_alone,1,1,1


In [13]:
cv.best_estimator_


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# testing a adaboost classifier

moc_ada = MultiOutputClassifier(AdaBoostClassifier())

pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc_ada)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_ada.fit(X_train.values, y_train.values)
y_pred = pipeline_ada.predict(X_test)
results3 = get_results(y_test, y_pred)
results3

Aggregated f_score: 0.940417064979
Aggregated precision: 0.939942834164
Aggregated recall: 0.947699454108


,Category,f_score,precision,recall
1,related,0.730453,0.733347,0.767013
2,request,0.888829,0.889443,0.896247
3,offer,0.99214,0.990809,0.993592
4,aid_related,0.749934,0.75876,0.75679
5,medical_help,0.913587,0.911783,0.92722
6,medical_products,0.952943,0.952196,0.959567
7,search_and_rescue,0.967213,0.965204,0.973299
8,security,0.974866,0.970459,0.981843
9,military,0.966093,0.964664,0.970705
10,child_alone,1,1,1


In [18]:
from sklearn.ensemble import AdaBoostClassifier

In [19]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f653c9b56a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [20]:
parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

cv_ada = GridSearchCV(pipeline_ada, parameters)

In [21]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = cv.predict(X_test)
results_ada = get_results(y_test, y_pred)
results_ada

Aggregated f_score: 0.92391795606
Aggregated precision: 0.929318325307
Aggregated recall: 0.940947852033


,Category,f_score,precision,recall
1,related,0.734885,0.765264,0.783033
2,request,0.847078,0.874637,0.87397
3,offer,0.992682,0.990259,0.995117
4,aid_related,0.73777,0.741751,0.742905
5,medical_help,0.891583,0.898526,0.922032
6,medical_products,0.930035,0.938659,0.95209
7,search_and_rescue,0.960129,0.94731,0.973299
8,security,0.974668,0.966414,0.983064
9,military,0.95403,0.957178,0.967958
10,child_alone,1,1,1


### 9. Export your model as a pickle file

In [23]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.